# Preparação da base de candidaturas cadastradas

[Voltar ao Índice](00_indice.ipynb)

Este notebook limpa e padroniza os dados de candidaturas LGBT cadastradas na plataforma do [VoteLGBT.org](https://votelgbt.org) para as eleições de 2022.

Fonte: <https://docs.google.com/spreadsheets/d/1lsRnBI4PDM6baI5SIG31-h0MjxF64YnFFNAXW018TVU>

In [2]:
import pandas as pd
import numpy as np
from pathlib import Path

import src.xavy.utils as xu
import src.xavy.dataframes as xd
import src.xavy.text as xt
import src.xavy.explore as xe

## Carregando e limpando os dados

### Carrega os dados

In [3]:
# Load data to be cleaned:
raw_df = pd.read_csv(Path('../dados/brutos/aux/candidaturas-2022_cadastro-voteLGBT_baixada-2022-12-04.csv'), skiprows=1, dtype=str)

### Limpa os dados

In [4]:
# Standardize strings:
clean_df = raw_df.copy()
for col in clean_df.columns:
    if col == 'Link':
        to_case = None
    else:
        to_case = 'upper'
    clean_df[col] = xd.std_string_series(clean_df[col], case=to_case)

# Turn column names to tag style:
new_col_names = [xt.text2tag(xt.keep_only_letters(t.replace('_', ' '))).replace('_de_', '_') for t in clean_df.columns]
col_renamer = dict(zip(clean_df.columns, new_col_names))
clean_df.rename(col_renamer, axis=1, inplace=True)

# Format numbers to be machine readable:
clean_df['numero_votos'] = clean_df['numero_votos'].str.replace('.', '', regex=False).astype(int)

# Add column SQ_CANDIDATO, extracted from link:
clean_df['sq_candidato'] = clean_df['link'].str.split('/').str.slice(-1).str.join('').astype(int)

# Reorder columns:
clean_df = clean_df[['sq_candidato'] + new_col_names]

### Exporta dados

In [6]:
# Para salvar os dados processados, descomente a linha abaixo (ela fica comentada para evitar sobrescrever o arquivo por acidente):
#clean_df.to_csv(Path('../dados/limpos/aux/candidaturas-2022_cadastro-voteLGBT_baixada-2022-12-04_clean.csv'), index=False)

## Testes com os dados limpos

### Carrega dados dos candidatos

In [ ]:
# Load TSE candidates' data for sanity checks:
cand_df = pd.read_csv(Path('../dados/limpos/consulta_cand/consulta_cand_2022_BRASIL.csv'), low_memory=False)

In [100]:
# Check properties of TSE candidates dataset:
assert (cand_df['NM_URNA_CANDIDATO'] == cand_df['NM_URNA_CANDIDATO'].str.upper()).all()
assert (cand_df['DS_CARGO'] == cand_df['DS_CARGO'].str.upper()).all()

### Testes sem cruzamento de dados

In [101]:
# Make sure SQ_CANDIDATO is unique:
assert xd.iskeyQ(clean_df[['sq_candidato']])

In [102]:
xe.checkMissing(clean_df)
print('# candidatos: {}'.format(len(clean_df)))

Colunas com valores faltantes:
Empty DataFrame
Columns: [coluna, N, %]
Index: []
# candidatos: 319


### Testes com cruzamento de dados

Se a base do TSE foi baixada antes do final das eleições, é possível que haja diferenças entre a base do VoteLGBT e a do TSE. Vamos verificar se elas não são muito grandes.

In [103]:
# Find candidates that are missing in our TSE data:
missing_cand = set(clean_df['sq_candidato']) - set(cand_df['SQ_CANDIDATO'])
print('Candidatos fora da base que temos do TSE:', missing_cand)
print('{}/{} ({:.1f}%)'.format(len(missing_cand), len(clean_df), len(missing_cand) / len(clean_df) * 100))

Candidatos fora da base que temos do TSE: {110001724934, 240001732331, 90001732912, 190001732307, 40001726396}
5/319 (1.6%)


Acima verificamos o número de candidatos VoteLGBT não encontrados. Esse número tem que ser pequeno.

In [104]:
# Select these candidates:
missing_cands_df = clean_df.loc[clean_df['sq_candidato'].isin(missing_cand)]

Abaixo verificamos os casos com informações divergentes. Elas não podem ser relevantes:

In [110]:
# Junta com dados do TSE:
sel_cand_cols = ['SG_UF', 'DS_CARGO', 'SG_PARTIDO', 'NM_URNA_CANDIDATO']
joined_df = clean_df.join(cand_df.set_index('SQ_CANDIDATO')[sel_cand_cols], on='sq_candidato', how='inner')

# Verifica diferenças de informação nos dados:
diff_uf      = joined_df['tse_uf'] != joined_df['SG_UF']
diff_cargo   = joined_df['tse_cargo'] != joined_df['DS_CARGO']
diff_partido = joined_df['tse_partido'] != joined_df['SG_PARTIDO'].str.upper()
diff_nome    = joined_df['tse_nome_urna'] != xd.std_string_series(joined_df['NM_URNA_CANDIDATO'])
# Candidaturas com dados diferentes:
display(joined_df.loc[diff_uf | diff_cargo | diff_partido, ['tse_nome_urna', 'tse_cargo', 'DS_CARGO', 'tse_uf', 'SG_UF', 'tse_partido', 'SG_PARTIDO']])
display(joined_df.loc[diff_nome, ['tse_nome_urna', 'NM_URNA_CANDIDATO']])

,tse_nome_urna,tse_cargo,DS_CARGO,tse_uf,SG_UF,tse_partido,SG_PARTIDO
63,RAFAELLA MACHADO,SENADOR (1O SUPLENTE),1º SUPLENTE,ES,ES,PSOL,PSOL
224,KELLY SILVA,DEPUTADO ESTADUAL,DEPUTADO FEDERAL,RS,RS,PDT,PDT
227,BRUNA E COLETIVO,DEPUTADO ESTADUAL,DEPUTADO ESTADUAL,RS,RS,PCDOB,PC do B


,tse_nome_urna,NM_URNA_CANDIDATO
0,DRA MICHELLE MELO,DR. MICHELLE MELO
6,FLORISMAR BANCADA AMAZÔNIDA,FLORISMAR FERREIRA
13,ANDERSON ROCHA,ANDRESON SANTOS
26,DRA. FABÍOLA MANSUR,DRA FABIOLA MANSUR
51,MARI VALENTIM,MARI VELENTIM
62,MULHERES DE TODAS AS LUTAS,FERNANDA PEREIRA
66,WALDIR PIRES,ENFERMEIRO WALDIR
81,CHICO COLETIVO GUARNICÊ,COLETIVO GUARNICÊ
103,COLETIVAS VOZES GERAIS/WALLACE,WALLACE
117,KARLA MELO COLETIVO SOMOS,KARLAMELO COLETIVO SOMOS
